# Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
from statistics import mean

# Import test data

In [ ]:
# Provide the file name of the test file
test_data = pd.read_csv('validation.csv')

In [ ]:
test_data.head()

# Load data and model objects

In [ ]:
with open('data_objects.pkl', 'rb') as input_file:
    data_list = pickle.load(input_file)

In [ ]:
with open('model_objects.pkl', 'rb') as input_file:
    model_list = pickle.load(input_file)

In [ ]:
with open('geohash_list.pkl', 'rb') as output_file:
    geohash_list = pickle.load(output_file)

# Data segmentation based on geolocation

In [ ]:
test_geohash_list = list(test_data.groupby('geohash').groups.keys())
test_gb = test_data.groupby('geohash')
test_mse_list = []

for i in tqdm(range(len(test_geohash_list))):
    try:
        index = geohash_list.index(test_geohash_list[i])
        print(test_geohash_list[i])
        print(index)
        print(geohash_list[index])
        test_data_segment = test_gb.get_group(test_geohash_list[i])
        data_list[index].set_test_data(test_data_segment)
        test_mse = model_list[index].test_model()
        test_mse_list.append(test_mse)
        print('Test MSE for {}: {}'.format(test_geohash_list[i], test_mse))
        
    except ValueError:
        print('{} is not included in the training data.'.format(test_geohash_list[i]))
        pass

# Report test MSE results

In [ ]:
sns.boxplot(y=test_mse_list)
plt.ylabel('Test MSE')
plt.title('Average Test MSE: {}'.format(mean(test_mse_list)))